# Brief Tour of MESS

MESS is a modular toolkit for exploring the exciting interface between machine
learning, electronic structure, and algorithms.

To begin our tour we build a single water molecule.
Each atom is represented an atomic number $Z_i$ and a position in Cartesian
coordinates $(x_i, y_i, z_i)$.  In MESS we collect atoms into a `Structure` and we
provide a few examples built by the `molecule` function.  
MESS is designed for interactive exploration so in a notebook environment a `Structure`
object will display a 3D visualisation

:::{note}
The following code cell will install MESS into the Google Colab runtime.
Select the 🚀 in the toolbar above to try this out!
:::

In [ ]:
import sys

if "google.colab" in sys.modules:
    !pip install mess-jax

In [3]:
from mess import molecule

mol = molecule("water")
mol

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Structure(atomic_number=i64[3](numpy), position=f64[3,3](numpy))

MESS represents electrons using the Linear Combination of Atomic Orbitals ([LCAO](https://en.wikipedia.org/wiki/Linear_combination_of_atomic_orbitals)) method.
We rely on the [Basis Set Exchange](https://www.basissetexchange.org/) project to
provide access to the full range of previously calculated Gaussian Type Orbital parameters.

In [4]:
from mess import basisset

basis_name = "6-31g"
basis = basisset(mol, basis_name)
basis

primitive    orbital    coefficient        norm  center                              lmn            alpha
-----------  ---------  -------------  ----------  ----------------------------------  -------  -----------
          0          0     0.00183107  454.227     [0.        0.        0.2201531]     [0 0 0]  5484.67
          1          0     0.0139502   109.735     [0.        0.        0.2201531]     [0 0 0]   825.235
          2          0     0.0684451    36.1918    [0.        0.        0.2201531]     [0 0 0]   188.047
          3          0     0.232714     13.9926    [0.        0.        0.2201531]     [0 0 0]    52.9645
          4          0     0.470193      5.93989   [0.        0.        0.2201531]     [0 0 0]    16.8976
          5          0     0.358521      2.66355   [0.        0.        0.2201531]     [0 0 0]     5.79964
          6          1    -0.110778      5.57815   [0.        0.        0.2201531]     [0 0 0]    15.5396
          7          1    -0.148026      1.86265   [0.        0.        0.2201531]     [0 0 0]     3.59993
          8          1     1.13077       0.720049  [0.        0.        0.2201531]     [0 0 0]     1.01376
          9          2     1             0.266956  [0.        0.        0.2201531]     [0 0 0]     0.270006
         10          3     0.0708743    43.9785    [0.        0.        0.2201531]     [1 0 0]    15.5396
         11          3     0.339753      7.06819   [0.        0.        0.2201531]     [1 0 0]     3.59993
         12          3     0.727159      1.44997   [0.        0.        0.2201531]     [1 0 0]     1.01376
         13          4     0.0708743    43.9785    [0.        0.        0.2201531]     [0 1 0]    15.5396
         14          4     0.339753      7.06819   [0.        0.        0.2201531]     [0 1 0]     3.59993
         15          4     0.727159      1.44997   [0.        0.        0.2201531]     [0 1 0]     1.01376
         16          5     0.0708743    43.9785    [0.        0.        0.2201531]     [0 0 1]    15.5396
         17          5     0.339753      7.06819   [0.        0.        0.2201531]     [0 0 1]     3.59993
         18          5     0.727159      1.44997   [0.        0.        0.2201531]     [0 0 1]     1.01376
         19          6     1             0.277432  [0.        0.        0.2201531]     [1 0 0]     0.270006
         20          7     1             0.277432  [0.        0.        0.2201531]     [0 1 0]     0.270006
         21          8     1             0.277432  [0.        0.        0.2201531]     [0 0 1]     0.270006
         22          9     0.0334946     6.41702   [ 0.         1.4539553 -0.8808013]  [0 0 0]    18.7311
         23          9     0.234727      1.55317   [ 0.         1.4539553 -0.8808013]  [0 0 0]     2.82539
         24          9     0.813757      0.510043  [ 0.         1.4539553 -0.8808013]  [0 0 0]     0.640122
         25         10     1             0.181381  [ 0.         1.4539553 -0.8808013]  [0 0 0]     0.161278
         26         11     0.0334946     6.41702   [ 0.        -1.4539553 -0.8808013]  [0 0 0]    18.7311
         27         11     0.234727      1.55317   [ 0.        -1.4539553 -0.8808013]  [0 0 0]     2.82539
         28         11     0.813757      0.510043  [ 0.        -1.4539553 -0.8808013]  [0 0 0]     0.640122
         29         12     1             0.181381  [ 0.        -1.4539553 -0.8808013]  [0 0 0]     0.161278

We now have all the pieces to run our first electronic structure simulation.
This is done in two steps:
* build a Hamiltonian by selecting a treatment for the quantum-mechanical exchange and correlation.
* find the molecular orbital coefficients $C$ that minimise the energy of this Hamiltonian

In the following we use the popular
[PBE exchange-correlation functional](https://doi.org/10.1103/PhysRevLett.77.3865)
of Density Functional Theory (DFT) to model the quantum-mechanical electron interactions.

By default the minimisation routine will be compiled by JAX and executed on any available
hardware accelerator (e.g. GPU/TPU).

In [5]:
from mess import minimise, Hamiltonian

H = Hamiltonian(basis, xc_method="pbe")
E, C, sol = minimise(H)
float(E)

-76.29898732997808

We can visualise the electron density $\rho(\mathbf{r})$ from the solution we found.
The electron cloud is approximately mickey mouse head shaped to use the technical term.

In [6]:
import py3Dmol
from mess.mesh import density, uniform_mesh
from mess.plot import plot_volume, plot_molecule

view = py3Dmol.view()
plot_molecule(view, mol)
mesh = uniform_mesh()
rho = density(basis, mesh, basis.density_matrix(C))
plot_volume(view, rho, mesh.axes)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Performance is one of several goals of the MESS project so lets measure how long the
energy minimisation takes

In [7]:
def mess_benchmark():
    E, C, _ = minimise(H)
    E, C = E.block_until_ready(), C.block_until_ready()


mess_time = %timeit -o mess_benchmark()

134 ms ± 711 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Before we get carried away profiling, we first make sure that the MESS simulation agrees with a standard
and well used DFT software package - [PySCF](https://pyscf.org/)

In [8]:
import numpy as np
from mess.interop import to_pyscf
from pyscf import dft, scf


scf_mol = to_pyscf(mol, basis_name)
s = dft.RKS(scf_mol, xc="pbe")
s.kernel()
np.allclose(s.energy_tot(), E)

/usr/local/lib/python3.10/dist-packages/pyscf/gto/mole.py:1286: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -76.2989811596524


True

The calculated energies match!...lets open a can of worms and measure the performance of
 the PySCF energy minimisation

In [9]:
pyscf_time = %timeit -o s.kernel()

converged SCF energy = -76.2989811596524
converged SCF energy = -76.2989811596524
converged SCF energy = -76.2989811596524
converged SCF energy = -76.2989811596523
converged SCF energy = -76.2989811596524
converged SCF energy = -76.2989811596523
converged SCF energy = -76.2989811596524
converged SCF energy = -76.2989811596524
2.12 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
pyscf_time.average / mess_time.average

15.896320044865424

We've measured a nearly 16X speedup...There are several gotchas of course that will be
explored in due time...stay tuned for more!

## Hartree-Fock

Hartree-Fock is a closely related method to the DFT solution found above that in MESS
is selected by passing `xc_method=hfx`

In [11]:
H = Hamiltonian(basis, xc_method="hfx")
E, C, sol = minimise(H)
float(E)

-75.9841721516878

The energy is a little higher than the DFT solution found earlier but at a significantly
reduced computational cost

In [12]:
%%timeit
minimise(H)

21.6 ms ± 358 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As another sanity check we make sure the calculated Hartree-Fock energy calculated by MESS agrees with PySCF

In [13]:
s = scf.RHF(scf_mol)
s.kernel()

/usr/local/lib/python3.10/dist-packages/pyscf/gto/mole.py:1286: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -75.9841721516933


-75.98417215169334

We hope you enjoyed your tour of MESS and welcome any feedback as
[github issues](https://github.com/graphcore-research/mess/issues) where we can continue the discussion.


:::{note}
For reproducibility we record the default accelerator (if any) used by JAX and the CPU architecture used to execute this notebook.
:::

In [14]:
import jax

jax.devices()[0].device_kind

'NVIDIA A100-SXM4-40GB'

In [15]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  12
  On-line CPU(s) list:   0-11
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               85
    Thread(s) per core:  2
    Core(s) per socket:  6
    Socket(s):           1
    Stepping:            7
    BogoMIPS:            4400.39
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd